In [289]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
file_path = "cleaned_data.xlsx"  
df = pd.read_excel(file_path)
print(df.head())
df.shape

  Learner SignUp DateTime                  Opportunity Id  \
0     2023-06-14 12:30:35  00000000-0GN2-A0AY-7XK8-C5FZPP   
1     2023-01-05 05:29:16  00000000-0GN2-A0AY-7XK8-C5FZPP   
2     2023-09-04 20:35:08  00000000-0GN2-A0AY-7XK8-C5FZPP   
3     2023-08-29 05:20:03  00000000-0GN2-A0AY-7XK8-C5FZPP   
4     2023-06-01 15:26:36  00000000-0GN2-A0AY-7XK8-C5FZPP   

                                    Opportunity Name Opportunity Category  \
0  Career Essentials: Getting Started with Your P...               Course   
1  Career Essentials: Getting Started with Your P...               Course   
2  Career Essentials: Getting Started with Your P...               Course   
3  Career Essentials: Getting Started with Your P...               Course   
4  Career Essentials: Getting Started with Your P...               Course   

  Opportunity End Date        First Name        Date of Birth  Gender  \
0  2024-06-29 18:52:39             Faria  2001-12-01 00:00:00  Female   
1  2024-06-29 18:52:39  

(8558, 16)

In [78]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Learner SignUp DateTime  8558 non-null   datetime64[ns]
 1   Opportunity Id           8558 non-null   object        
 2   Opportunity Name         8558 non-null   object        
 3   Opportunity Category     8558 non-null   object        
 4   Opportunity End Date     8558 non-null   datetime64[ns]
 5   First Name               8558 non-null   object        
 6   Date of Birth            8558 non-null   object        
 7   Gender                   8558 non-null   object        
 8   Country                  8558 non-null   object        
 9   Institution Name         8558 non-null   object        
 10  Current/Intended Major   8558 non-null   object        
 11  Entry created at         8558 non-null   datetime64[ns]
 12  Status Description       8558 non-

In [291]:
string_columns = ["First Name", "Country", "Institution Name", "Current/Intended Major", "Opportunity Name", "Opportunity Category", "Status Description", "Gender"]
df[string_columns] = df[string_columns].astype("string")

print(df.dtypes)

Learner SignUp DateTime    datetime64[ns]
Opportunity Id                     object
Opportunity Name           string[python]
Opportunity Category       string[python]
Opportunity End Date       datetime64[ns]
First Name                 string[python]
Date of Birth                      object
Gender                     string[python]
Country                    string[python]
Institution Name           string[python]
Current/Intended Major     string[python]
Entry created at           datetime64[ns]
Status Description         string[python]
Status Code                         int64
Apply Date                 datetime64[ns]
Opportunity Start Date     datetime64[ns]
dtype: object


In [324]:
print("\n1. DATE/TIME FEATURES:")

# Convert date columns to datetime
date_columns = ['Learner SignUp DateTime', 'Date of Birth', 'Apply Date',
                'Opportunity End Date', 'Opportunity Start Date', 'Entry created at']

for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# 1. Raw Age (in years, integer)
df['Age'] = ((datetime.now() - df['Date of Birth']).dt.days / 365.25).astype(int)

# 2. Keep a copy of Age before scaling
df['Current_Age'] = df['Age']

# 3. Standardize Current_Age
scaler = StandardScaler()
df['Current_Age'] = scaler.fit_transform(df[['Current_Age']])

# Age groups
df['Age_Group'] = pd.cut(df['Age'],
                         bins=[0, 20, 25, 30, 35, 100],
                         labels=['Teen', 'Young_Adult', 'Adult', 'Mid_Adult', 'Senior'])

# Signup to Apply gap
df['Signup_to_Apply_Days'] = (df['Apply Date'] - df['Learner SignUp DateTime']).dt.days
df['Course_Duration_Days'] = (df['Opportunity End Date'] - df['Opportunity Start Date']).dt.days

# Signup breakdown
df['Signup_Month'] = df['Learner SignUp DateTime'].dt.month
df['Signup_Weekday'] = df['Learner SignUp DateTime'].dt.day_name()
df['Signup_Hour'] = df['Learner SignUp DateTime'].dt.hour
df['Is_Weekend_Signup'] = df['Learner SignUp DateTime'].dt.weekday >= 5

# Quick applicant flag
df['Quick_Applicant'] = (df['Signup_to_Apply_Days'] <= 1).astype(int)

print("Date features created:")
print("   - Age, Current_Age, Age_Group")
print("   - Signup_to_Apply_Days, Course_Duration_Days")
print("   - Signup_Month, Signup_Weekday, Signup_Hour")
print("   - Is_Weekend_Signup, Quick_Applicant")

df['Invalid_Application_Timing'] = df['Signup_to_Apply_Days'] < 0


1. DATE/TIME FEATURES:
Date features created:
   - Age, Current_Age, Age_Group
   - Signup_to_Apply_Days, Course_Duration_Days
   - Signup_Month, Signup_Weekday, Signup_Hour
   - Is_Weekend_Signup, Quick_Applicant


In [295]:
print("\n2. CATEGORICAL FEATURES:")

df['Gender_Encoded'] = df['Gender'].map({
    'Male': 1, 'Female': 2, 'Other': 3, "Don't want to specify": 4
})

country_mapping = {
    'United States': 'North_America',
    'Canada': 'North_America',
    'India': 'South_Asia',
    'Pakistan': 'South_Asia',
    'Bangladesh': 'South_Asia',
    'Nigeria': 'Africa',
    'Ghana': 'Africa',
    'Egypt': 'Africa',
    'United Kingdom': 'Europe',
    'Germany': 'Europe'
}

df['Region'] = df['Country'].map(country_mapping).fillna('Other')

def categorize_major(major):
    if pd.isna(major):
        return "Other"
    major_lower = major.lower()

    if any(word in major_lower for word in ["computer", "software", "cyber", "data", "information", "ai", "machine learning", "it"]):
        return "Tech"
    elif any(word in major_lower for word in ["business", "finance", "marketing", "management", "commerce", "accounting"]):
        return "Business"
    elif any(word in major_lower for word in ["health", "medicine", "nursing", "pharmacy", "biomedical", "public health", "medical", "clinical"]):
        return "Health"
    else:
        return "Other"

df["Major_Category"] = df["Current/Intended Major"].apply(categorize_major)

# Status success indicator
success_statuses = ['Started', 'Team Allocated', 'Rewards Award']
df['Is_Successful'] = df['Status Description'].isin(success_statuses).astype(int)

print("Categorical features created:")
print("   - Gender_Encoded")
print("   - Region (country grouping)")
print("   - Major_Category")
print("   - Is_Successful")


2. CATEGORICAL FEATURES:
Categorical features created:
   - Gender_Encoded
   - Region (country grouping)
   - Major_Category
   - Is_Successful


In [297]:
print("\n3. BEHAVIORAL FEATURES:")

df['Same_Day_Application'] = (df['Signup_to_Apply_Days'] == 0).astype(int)
df['Late_Applicant'] = (df['Signup_to_Apply_Days'] > 30).astype(int)

opportunity_counts = df['Opportunity Name'].value_counts()
df['Opportunity_Popularity'] = df['Opportunity Name'].map(opportunity_counts)
df['Popular_Opportunity'] = (df['Opportunity_Popularity'] > df['Opportunity_Popularity'].median()).astype(int)

print("Behavioral features created:")
print("   - Same_Day_Application, Late_Applicant")
print("   - Opportunity_Popularity, Popular_Opportunity")


3. BEHAVIORAL FEATURES:
Behavioral features created:
   - Same_Day_Application, Late_Applicant
   - Opportunity_Popularity, Popular_Opportunity


In [299]:
print("\n4. INTERACTION FEATURES:")

df['Age_Major_Combo'] = df['Age_Group'].astype(str) + '_' + df['Major_Category']

df['Country_Gender'] = df['Country'] + '_' + df['Gender']

df['Region_Opportunity'] = df['Region'] + '_' + df['Opportunity Category']

print("Interaction features created:")
print("   - Age_Major_Combo")
print("   - Country_Gender")
print("   - Region_Opportunity")


4. INTERACTION FEATURES:
Interaction features created:
   - Age_Major_Combo
   - Country_Gender
   - Region_Opportunity


In [301]:
print("\n5. FEATURE SCALING:")

numerical_cols = ['Current_Age', 'Course_Duration_Days',
                  'Opportunity_Popularity', 'Signup_to_Apply_Days']

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print("Features scaled using StandardScaler!")
print(f"   - Age mean: {df['Current_Age'].mean():.2f}, std: {df['Current_Age'].std():.2f}")
print(f"   - Duration mean: {df['Course_Duration_Days'].mean():.2f}, std: {df['Course_Duration_Days'].std():.2f}")
print(f"   - Popularity mean: {df['Opportunity_Popularity'].mean():.2f}, std: {df['Opportunity_Popularity'].std():.2f}")


5. FEATURE SCALING:
Features scaled using StandardScaler!
   - Age mean: 0.00, std: 1.00
   - Duration mean: -0.00, std: 1.00
   - Popularity mean: -0.00, std: 1.00


In [303]:
print("\n6. ENGAGEMENT SCORE:")

df['Engagement_Score'] = (
    df['Quick_Applicant'] * 0.25 +
    df['Same_Day_Application'] * 0.15 +
    df['Popular_Opportunity'] * 0.20 +
    df['Is_Successful'] * 0.40
)

df['Engagement_Level'] = pd.cut(
    df['Engagement_Score'],
    bins=[-0.01, 0.3, 0.6, 1.01],  # extend range slightly
    labels=['Low', 'Medium', 'High'],
    include_lowest=True
)

print("Engagement Score created:")
print(f"   - Score range: {df['Engagement_Score'].min():.2f} to {df['Engagement_Score'].max():.2f}")
print(f"   - High engagement: {(df['Engagement_Level'] == 'High').sum()} students")
print(f"   - Medium engagement: {(df['Engagement_Level'] == 'Medium').sum()} students")
print(f"   - Low engagement: {(df['Engagement_Level'] == 'Low').sum()} students")


6. ENGAGEMENT SCORE:
Engagement Score created:
   - Score range: 0.00 to 1.00
   - High engagement: 3835 students
   - Medium engagement: 2251 students
   - Low engagement: 2472 students


In [305]:
# 1. Apply vs Start Gap
df["Apply_vs_Start_Gap"] = (
    df["Apply Date"] - df["Opportunity Start Date"]
).dt.days

# --- User Behavior Features ---
# 2. Applications per User
df["Applications_per_User"] = df.groupby("First Name")["Opportunity Id"].transform("count")

# 3. Unique Categories per User
df["Unique_Categories_per_User"] = df.groupby("First Name")["Opportunity Category"].transform("nunique")

# --- Geomorphic Features ---
# 4. Region + Age Group feature
df['Region_AgeGroup'] = df["Region"].astype(str) + "_" + df["Age_Group"].astype(str)

# --- Composition Features ---
# 7. Applications for Opportunity (Composition Feature)
df["Applications_for_Opportunity"] = df.groupby("Opportunity Id")["First Name"].transform("count")

# 5. Region Normalized Applications
df["Region_Normalized_Applications"] = (
    df.groupby("Region")["Applications_per_User"]
    .transform(lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6))
)

# 6. User Success Rate (Application vs Win)
if "Win" in df.columns: # checking if 'Win' column exists to mark successful applications
    df["User_Success_Rate"] = (
        df.groupby("First Name")["Win"].transform("sum") / (df["Applications_per_User"] + 1e-6)
    )
else:
    df["User_Success_Rate"] = np.nan  # placeholder if 'Win' column not available

# Select only the feature columns for export in the specified order
feature_columns = [
    'Apply_vs_Start_Gap',
    'Applications_per_User',
    'Unique_Categories_per_User',
    'Region_AgeGroup',
    'Applications_for_Opportunity',
    'Region_Normalized_Applications',
    'User_Success_Rate'
]

# Add 'Win' column if it exists
if "Win" in df.columns:
    feature_columns.append('Win')

# Create a new DataFrame with only the feature columns
df_features = df[feature_columns].copy()

In [271]:
print(f"Total number of columns: {len(df.columns)}")

Total number of columns: 47


In [326]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 47 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Learner SignUp DateTime         8558 non-null   datetime64[ns]
 1   Opportunity Id                  8558 non-null   object        
 2   Opportunity Name                8558 non-null   string        
 3   Opportunity Category            8558 non-null   string        
 4   Opportunity End Date            8558 non-null   datetime64[ns]
 5   First Name                      8558 non-null   string        
 6   Date of Birth                   8558 non-null   datetime64[ns]
 7   Gender                          8558 non-null   string        
 8   Country                         8558 non-null   string        
 9   Institution Name                8558 non-null   string        
 10  Current/Intended Major          8558 non-null   string        
 11  Entr